- Make a chain that is specialized in writing Haikus about programming languages and another one that is specialized in explaining Haikus.  
  프로그래밍 언어에 대한 시를 쓰는 데 특화된 체인과 시를 설명하는 데 특화된 체인을 만드세요.
- Chain the two chains together using LCEL.  
  LCEL을 사용해 두 체인을 서로 연결합니다.
- The final chain should receive the name of a programming language and it should reply with the Haiku and its explanation.  
  최종 체인은 프로그래밍 언어의 이름을 받고 시와 그 설명으로 응답해야 합니다.
- Use "gpt-3.5-turbo" as the model and ChatPromptTemplate for your prompts.  
  모델로는 "gpt-3.5-turbo"를 사용하고 프롬프트에는 ChatPromptTemplate을 사용하세요.


In [ ]:
# ChatPromptTemplate : Create a template from messages (PromptTemplate : from string)

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.2,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [2]:
poet_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a creative and eloquent poet who specializes in writing poems about programming languages. Your style can be whimsical, metaphorical, or technical, blending the beauty of poetry with the logic of programming. Ensure your poems are vivid, imaginative, and capture the essence of the requested programming language.",
        ),
        # ("ai", ""),
        (
            "human",
            "Write a poem about the programming language {language}. Highlight its unique characteristics, quirks, and strengths. You can use metaphors, analogies, and creative language to make the poem engaging and insightful. \
            For example: \
            - If the language is Python, you might emphasize simplicity and readability. \
            - If it's C++, you might capture its power and complexity.",
        ),
    ]
)

poet_chain = poet_prompt | chat

In [3]:
explainer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an insightful poetry analyst who specializes in interpreting poems about programming languages. Your role is to analyze the meaning, metaphors, and stylistic choices within a poem. Provide clear and thoughtful explanations, highlighting how the poem reflects the characteristics of the programming language it is about. Offer interpretations that are easy to understand, while maintaining depth and nuance.",
        ),
        (
            "human",
            "{poem}. Respond is `the input poem` and `your explanation about that`. in Korean.",
        ),
    ]
)

explainer_chain = explainer_prompt | chat

In [4]:
chain = {"poem": poet_chain} | explainer_chain

chain.invoke({"language": "python"})

In the heart of the digital jungle, where serpents gracefully glide,
Lies a language called Python, where beauty and logic coincide.
With syntax so clean, like a river flowing free,
It enchants all who dare to code, from novice to prodigy.

Like a python coiled, it waits patiently to strike,
With indentation as its weapon, code elegant and light.
No curly braces clutter, no semicolons to bind,
Just spaces and tabs, in perfect unison aligned.

Its libraries vast, like a treasure trove untold,
NumPy, Pandas, and more, a programmer's stronghold.
With simplicity as its cloak, and readability its song,
Python dances through algorithms, swift and strong.

From data science to web apps, it conquers every field,
A versatile companion, its power unconcealed.
Like a magician's wand, it weaves spells with ease,
Turning lines of code into wonders that please.

So embrace the Pythonic way, let its magic unfold,
In the realm of programming, a story waiting to be told.
For in the world of languages, 

AIMessageChunk(content='`the input poem`\n\n"디지털 정글의 심장 속, 뱀들이 우아하게 휘몰아치는 곳,\\n아름다움과 논리가 공존하는 언어 파이썬이 놓여 있네.\\n강이 흐르는 듯이 맑은 문법,\\n초보자부터 천재까지 도전하는 이들을 매혹시키네.\\n\\n말뚝처럼 감아진 파이썬, 차분히 기회를 기다리네,\\n들여쓰기가 무기인, 우아하고 가벼운 코드.\\n중괄호로 얽힌 코드가 없고, 세미콜론이 묶이지 않네,\\n그저 공백과 탭이 완벽하게 일치하는 것 뿐이네.\\n\\n넘파이, 판다스 등 무궁무진한 라이브러리,\\n프로그래머의 요새.\\n단순함이 망토, 가독성이 노래인,\\n파이썬은 알고리즘을 빠르고 강하게 춤추네.\\n\\n데이터 과학부터 웹 앱까지, 모든 분야를 정복하며,\\n다재다능한 동반자, 그 힘 숨기지 않네.\\n마술사의 지팡이처럼, 쉽게 주문을 걸어넣네,\\n코드 한 줄을 기적으로 만들어내네.\\n\\n파이썬의 방식을 받아들여라, 그 마법을 펼쳐라,\\n프로그래밍의 영역에서 들려오는 이야기.\\n언어의 세계에서, 복잡함이 넘쳐나는 곳에서,\\n파이썬은 빛나네, 정말 깊이 감동적인 보석이네."\n\n`your explanation about that`\n\n이 시는 파이썬이라는 프로그래밍 언어를 표현한 것으로, 파이썬의 특징과 성격을 잘 반영하고 있습니다. 시인은 파이썬의 간결하고 아름다운 문법을 강조하며, 들여쓰기를 중요한 요소로 언급하여 파이썬의 가독성과 우아함을 표현했습니다. 또한 넘파이, 판다스와 같은 다양한 라이브러리를 보물창고로 비유하여 파이썬의 다양한 기능과 활용성을 강조했습니다.\n\n시는 파이썬이 데이터 과학부터 웹 애플리케이션까지 모든 분야에서 활약하며 강력한 힘을 발휘하는 다재다능한 언어임을 보여줍니다. 마치 마술사의 지팡이처럼 파이썬이 코드를 마법으로 변환시키는 모습을 통해 파이썬의 효율성과 능력을 감탄스럽게 표현했습니다.\n\n시는 파이썬의 매력과 힘을 감상적으로 표현하면서, 프로그래밍 언어로서의